In [20]:
import pandas as pd

In [21]:
ema = pd.read_csv("/home/donsson/proyectos/MODELO ABASTECIMIENTO/ema_mio202536.csv")

In [22]:
ema["Venta costo EMA"] = ema["EMA"]*ema["producto_costo_unitario"]

import pandas as pd

ema = ema.copy()

# 1. Ordenar por "Venta costo EMA" descendente (así se acumula de mayor a menor)
ema = ema.sort_values("Venta costo EMA", ascending=False).reset_index(drop=True)

# 2. Calcular el total de "Venta costo EMA"
total_venta_costo = ema["Venta costo EMA"].sum()

# 3. Calcular porcentaje individual de cada producto
ema["Venta costo %"] = ema["Venta costo EMA"] / total_venta_costo

# 4. Calcular el acumulado
ema["Acumulado costo"] = ema["Venta costo EMA"].cumsum()

# 5. Calcular el porcentaje acumulado
ema["Acumulado costo %"] = ema["Acumulado costo"] / total_venta_costo

# 6. (opcional) Renombrar columnas para que se vean como en tu reporte Odoo
ema_def = ema.rename(columns={
    "año": "Año",
    "semana_num": "Semana",
    "EMA": "Ema",
    "producto_costo_unitario": "Costo unitario",
    "Venta costo EMA": "Venta costo EMA"
})


In [23]:
ema.query("(store_name == 'SUCURSAL BARRANQUILLA') & (product_ref == 'DAB02570025')")


,Unnamed: 0,store_name,product_ref,año,semana_num,EMA,demanda_ajustada,producto_costo_unitario,Venta costo EMA,Venta costo %,Acumulado costo,Acumulado costo %
65,343537,SUCURSAL BARRANQUILLA,DAB02570025,2025.0,36,45.69405,44.0,13507.71,617221.976125,0.001978,7.632988e+07,0.244565


In [32]:
ema_def = ema_def[["store_name","product_ref","Año","Semana","Ema","demanda_ajustada","Costo unitario","Venta costo EMA","Venta costo %","Acumulado costo %"]]

ema_costo = ema_def[ema_def["store_name"]=="SUCURSAL BARRANQUILLA"].sort_values(by=("Ema"),ascending=False).head(10)

ema_costo.head()

,store_name,product_ref,Año,Semana,Ema,demanda_ajustada,Costo unitario,Venta costo EMA,Venta costo %,Acumulado costo %
65,SUCURSAL BARRANQUILLA,DAB02570025,2025.0,36,45.694050,44.0,13507.71,6.172220e+05,0.001978,0.244565
18,SUCURSAL BARRANQUILLA,BLS00037125,2025.0,36,37.222562,34.0,32797.97,1.220824e+06,0.003912,0.112893
152,SUCURSAL BARRANQUILLA,DAB02772025,2025.0,36,36.339721,48.0,10680.97,3.881435e+05,0.001244,0.377250
209,SUCURSAL BARRANQUILLA,DAB14570025,2025.0,36,33.086309,60.0,9774.98,3.234180e+05,0.001036,0.442134
84,SUCURSAL BARRANQUILLA,BCS00035125,2025.0,36,26.137933,14.0,20638.72,5.394535e+05,0.001728,0.279415
